In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import shutil
import random

images_dir = '/content/drive/MyDrive/ToothNumber_TaskDataset/images'
labels_dir = '/content/drive/MyDrive/ToothNumber_TaskDataset/labels'

images = sorted([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))])
labels = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

assert all(os.path.splitext(img)[0] == os.path.splitext(lbl)[0]
           for img, lbl in zip(images, labels)), "Mismatch between images and labels."

for split in ['train', 'val', 'test']:
    os.makedirs(f'/content/dataset/{split}/images', exist_ok=True)
    os.makedirs(f'/content/dataset/{split}/labels', exist_ok=True)

data = list(zip(images, labels))
random.shuffle(data)
n = len(data)
train_data = data[:int(0.8 * n)]
val_data = data[int(0.8 * n):int(0.9 * n)]
test_data = data[int(0.9 * n):]

def move_files(split_data, split_name):
    for img, lbl in split_data:
        shutil.copy(os.path.join(images_dir, img), f'/content/dataset/{split_name}/images/{img}')
        shutil.copy(os.path.join(labels_dir, lbl), f'/content/dataset/{split_name}/labels/{lbl}')

move_files(train_data, 'train')
move_files(val_data, 'val')
move_files(test_data, 'test')

print("Dataset splitting done!")


Dataset splitting done!


In [3]:
yaml_content = """
train: /content/dataset/train/images
val: /content/dataset/val/images
test: /content/dataset/test/images

names:
  0: Canine (13)
  1: Canine (23)
  2: Canine (33)
  3: Canine (43)
  4: Central Incisor (21)
  5: Central Incisor (41)
  6: Central Incisor (31)
  7: Central Incisor (11)
  8: First Molar (16)
  9: First Molar (26)
  10: First Molar (36)
  11: First Molar (46)
  12: First Premolar (14)
  13: First Premolar (34)
  14: First Premolar (44)
  15: First Premolar (24)
  16: Lateral Incisor (22)
  17: Lateral Incisor (32)
  18: Lateral Incisor (42)
  19: Lateral Incisor (12)
  20: Second Molar (17)
  21: Second Molar (27)
  22: Second Molar (37)
  23: Second Molar (47)
  24: Second Premolar (15)
  25: Second Premolar (25)
  26: Second Premolar (35)
  27: Second Premolar (45)
  28: Third Molar (18)
  29: Third Molar (28)
  30: Third Molar (38)
  31: Third Molar (48)
"""

with open('/content/dataset/data.yaml', 'w') as f:
    f.write(yaml_content.strip())

print("data.yaml created successfully!")


data.yaml created successfully!


In [4]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO

# Load pretrained YOLOv8 small model
model = YOLO('yolov8s.pt')

# Train on your dataset using the data.yaml from Step 2
model.train(data='/content/dataset/data.yaml', epochs=100, imgsz=640, batch=16, name='oralvis_tooth_detection')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b16b01f9040>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [14]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/oralvis_tooth_detection/weights/best.pt')

metrics = model.val(data='/content/dataset/data.yaml')

summary = metrics.results_dict

print("Keys in results_dict:", summary.keys())

print(f"Precision: {summary.get('metrics/precision(B)', 'N/A'):.4f}")
print(f"Recall: {summary.get('metrics/recall(B)', 'N/A'):.4f}")
print(f"mAP@50: {summary.get('metrics/mAP50(B)', 'N/A'):.4f}")
print(f"mAP@50-95: {summary.get('metrics/mAP50-95(B)', 'N/A'):.4f}")



Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,137,968 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1392.2±704.2 MB/s, size: 75.0 KB)
val: Scanning /content/dataset/val/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 63782.0it/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 34, len(boxes) = 1346. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.96it/s 4.2s
                   all         50       1346      0.866      0.874      0.902       0.64
           Canine (13)         45         45      0.855      0.911      0.924      0.677
           Canine (23)         45         45  

In [16]:
import os
from ultralytics import YOLO

# Load your best trained model weights
model = YOLO('/content/runs/detect/oralvis_tooth_detection/weights/best.pt')

output_dir = '/content/runs/predict/oralvis_samples'
os.makedirs(output_dir, exist_ok=True)

test_images_dir = '/content/dataset/test/images'

for img_file in os.listdir(test_images_dir):
    if img_file.endswith(('.jpg', '.png')):
        img_path = os.path.join(test_images_dir, img_file)
        model.predict(source=img_path, save=True, project=output_dir, name='results', exist_ok=True)



image 1/1 /content/dataset/test/images/cate8-00267_jpg.rf.8303c47ab0c59b7d7ae202f0041fcf20.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 1 Central Incisor (11), 1 First Molar (16), 1 First Molar (26), 1 First Molar (36), 1 First Molar (46), 1 First Premolar (14), 1 First Premolar (34), 1 First Premolar (44), 1 First Premolar (24), 1 Lateral Incisor (22), 1 Lateral Incisor (32), 1 Lateral Incisor (42), 1 Lateral Incisor (12), 1 Second Molar (17), 1 Second Molar (27), 1 Second Molar (37), 2 Second Molar (47)s, 1 Second Premolar (15), 1 Second Premolar (25), 1 Second Premolar (35), 1 Second Premolar (45), 1 Third Molar (28), 1 Third Molar (38), 16.3ms
Speed: 1.8ms preprocess, 16.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/predict/oralvis_samples/results

image 1/1 /content/dataset/test/images/23f1d012-20240628-115132889.jpg: 640x640 1 

In [20]:
import numpy as np
from sklearn.cluster import KMeans

def anatomical_postprocessing(detections, img_width, img_height):
    if len(detections) == 0:
        return []

    points = np.array([[d['bbox'][0] * img_width, d['bbox'][1] * img_height] for d in detections])
    y_coords = points[:, 1].reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(y_coords)
    labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_.flatten()
    upper_cluster = np.argmin(cluster_centers)

    for i, d in enumerate(detections):
        d['arch'] = 'upper' if labels[i] == upper_cluster else 'lower'

    mid_x = img_width / 2
    for d in detections:
        x_center = d['bbox'][0] * img_width
        if d['arch'] == 'upper':
            d['quadrant'] = 1 if x_center >= mid_x else 2
        else:
            d['quadrant'] = 4 if x_center >= mid_x else 3

    output = []
    for quad in [1, 2, 3, 4]:
        quad_dets = [d for d in detections if d['quadrant'] == quad]
        quad_dets.sort(key=lambda x: x['bbox'][0])
        for pos, det in enumerate(quad_dets, start=1):
            det['fdi_number'] = int(f"{det['quadrant']}{pos}")
            output.append(det)

    return output


# --- Sample detection input (mock data) ---
# Format: class_id, normalized bbox = [x_center, y_center, width, height]
sample_detections = [
    {'class_id': 7, 'bbox': [0.85, 0.25, 0.05, 0.07]},  # Upper right, near right
    {'class_id': 4, 'bbox': [0.75, 0.27, 0.04, 0.06]},
    {'class_id': 16, 'bbox': [0.25, 0.24, 0.04, 0.05]}, # Upper left
    {'class_id': 25, 'bbox': [0.20, 0.26, 0.04, 0.06]},
    {'class_id': 10, 'bbox': [0.80, 0.75, 0.06, 0.08]}, # Lower right
    {'class_id': 3, 'bbox': [0.85, 0.78, 0.05, 0.07]},
    {'class_id': 1, 'bbox': [0.15, 0.76, 0.05, 0.06]},  # Lower left
    {'class_id': 6, 'bbox': [0.25, 0.73, 0.04, 0.05]}
]

img_w, img_h = 1024, 512

processed = anatomical_postprocessing(sample_detections, img_w, img_h)

print("Processed Detections with Reassigned FDI Numbers:")
for det in processed:
    print(f"Class ID={det['class_id']}, Arch={det['arch']}, Quadrant={det['quadrant']}, "
          f"FDI Number={det['fdi_number']}, BBox(center_x={det['bbox'][0]:.2f}, center_y={det['bbox'][1]:.2f})")


Processed Detections with Reassigned FDI Numbers:
Class ID=4, Arch=upper, Quadrant=1, FDI Number=11, BBox(center_x=0.75, center_y=0.27)
Class ID=7, Arch=upper, Quadrant=1, FDI Number=12, BBox(center_x=0.85, center_y=0.25)
Class ID=25, Arch=upper, Quadrant=2, FDI Number=21, BBox(center_x=0.20, center_y=0.26)
Class ID=16, Arch=upper, Quadrant=2, FDI Number=22, BBox(center_x=0.25, center_y=0.24)
Class ID=1, Arch=lower, Quadrant=3, FDI Number=31, BBox(center_x=0.15, center_y=0.76)
Class ID=6, Arch=lower, Quadrant=3, FDI Number=32, BBox(center_x=0.25, center_y=0.73)
Class ID=10, Arch=lower, Quadrant=4, FDI Number=41, BBox(center_x=0.80, center_y=0.75)
Class ID=3, Arch=lower, Quadrant=4, FDI Number=42, BBox(center_x=0.85, center_y=0.78)
